# Data Release Example Scripts: GP Flux Measurement

This notebooks illustrates how the Galactic Plane analysis can be reproduced with the provided data release.

## Import necessary python libraries

In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats, optimize
from scipy.interpolate import UnivariateSpline
import healpy as hp

## Define some global settings

In [ ]:

# directory that contains the data release files
data_dir = '/data/user/mhuennefeld/data/analyses/DNNCascadeCodeReview/unblinding_checks/plots/data_release/create_data/DataFluxMeasurement/'


## Load event-level data from data release

The event-level data contains the contributions to each of the likelihood-ratio terms and it is provided via csv files. We will load these via the python library pandas.

In [ ]:

df_events = pd.read_csv(os.path.join(data_dir, 'event_contributions_gp.csv'))
df_events

## Define Likelihood-ratio and optimization functions

As stated in the paper, the "signal-subtracted" likelihood is given by:

$
\begin{equation}
 \mathcal{L} (n_s, \gamma) = \prod_{i=1}^N \frac{n_s}{N} S_i(\gamma, \delta_i, \alpha_i, \sigma_i, E_i) + \tilde{D_i} (\delta_i, E_i) - \frac{n_s}{N}\tilde{S_i} (\delta_i, E_i) 
    \label{eq:ssllh}
\end{equation}
$

with the PDF $\tilde{D}$ obtained from the scrambled experimental data and 
$\tilde{S}$ the scrambled signal PDF as obtained from MC.

The test-statistic (TS) is defined by the ratio of the maximized likelihood and the likelihood for the null hypothesis:

$
\begin{equation}
TS = -2 \ln \frac{ \mathcal{L} (n_s = 0)}{ \mathcal{L}(\hat{n}_s,\gamma)}
    \label{eq:TS}
\end{equation}
$

Inserting the definitions for the likelihood in the above TS equation (and omitting the function arguments) results in

$
\begin{equation}
TS(n_s) = -2 \ln \frac{
        \prod_{i=1}^N \frac{n_s}{N} S_i + \tilde{D_i} - \frac{n_s}{N}\tilde{S_i}
    }{ 
       \prod_{i=1}^N \tilde{D_i}
    }
   = -2 \ln \prod_{i=1}^N  \left( \frac{n_s}{N} \cdot 
        \frac{S_i -\tilde{S_i}}{\tilde{D_i}}
    + 1
    \right)
   = -2 \sum_{i=1}^N \ln \left( \frac{n_s}{N} \cdot 
        \frac{S_i -\tilde{S_i}}{\tilde{D_i}}
    + 1
    \right)
\end{equation}
$

Directly optimizing the test-statistic as a function of $n_s$ is equivalent to first optimizing the likelihood $\mathcal L (n_s)$ and subsquently computing the test-statistic. The analysis software directly optimizes the test-statistic as a function of $n_s$. This has the added benefit that individual analysis PDFs for signal and background only need to be defined as ratios.
Furthermore, the PDFs can be split up into an energy and spacial term. We can therefore also write:

$
\begin{equation}
TS(n_s) 
   = -2 \sum_{i=1}^N \ln \left( \frac{n_s}{N} \cdot 
        \left(
            \frac{S_i^\mathrm{space}}{\tilde{D_i}^\mathrm{space}}
            - \frac{\tilde{S_i}^\mathrm{space}}{\tilde{D_i}^\mathrm{space}}
        \right) \cdot 
        \frac{S_i^\mathrm{energy}}{\tilde{D_i}^\mathrm{energy}}
    + 1
    \right)
   %= -2 \sum_{i=1}^N \ln \left( \frac{n_s}{N} \cdot 
   %     \frac{\left(S_i^\mathrm{space} -\tilde{S_i}^\mathrm{space}\right) \cdot S_i^\mathrm{energy}}{\tilde{D_i}^\mathrm{space}\cdot\tilde{D_i}^\mathrm{energy}}
   % + 1
   % \right)
\end{equation}
$

Here we used the fact that the signal energy PDF remains the same for scrambled and un-scrambled events, e.g. $S_i^\mathrm{energy}$ = $\tilde{S_i}^\mathrm{energy}$.

The individual ratio terms for each event are provided in the data release. In terms of the column names in the data file, this results in :

$
\begin{equation}
TS(n_s) 
   = -2 \sum_{i=1}^N \ln \left( \frac{n_s}{N} \cdot 
        \left(
            \mathrm{SoB\_space}_i
            - \mathrm{SoB\_space\_ss}_i
        \right) \cdot 
        \mathrm{SoB\_energy}_i
    + 1
    \right)
\end{equation}
$
We will use this equation to define the python function `get_test_statistic`.

In [ ]:

def get_test_statistic(ns, N, SoB_space, SoB_space_ss, SoB_energy):
    """Compute test-statistic
    
    Parameters
    ----------
    ns : float
        The number of signal events.
    N : int
        The total number of events.
    SoB_space : array_like
        The ratio of the signal and background space PDF.
    SoB_space_ss : array_like
        The ratio of the scrambled signal and background space PDF.
    SoB_energy : array_like
        The ratio of the signal and background energy PDF.
    
    Returns
    -------
    float
        The test-statistic of the maximized likelihood ratio.
    """
    ts = np.sum(2*np.log((SoB_space - SoB_space_ss) * SoB_energy * ns/N + 1))
    return ts


We can now optimize the test-statistic. We will use scipy for the optimization and we will loop through each of the GP models. The best fit $\hat{n}_s$ is then saved in the dictionary `res_dict`.

In [ ]:
# define the function we need to minimize
def get_negative_test_statistic(*args, **kwargs):
    return -get_test_statistic(*args, **kwargs)

# define a dictionary which will hold our results
res_dict = {}

# loop through each GP model
for key in ['pi0', 'kra5', 'kra50']:
    
    # Minimize negative test-statistic function via scipy
    print('Now optimizing the test-statistic for the GP model {}'.format(key))
    result = optimize.minimize(
        get_negative_test_statistic, 
        x0=0, 
        args=(
            len(df_events),  # N
            df_events[key + '_SoB_space'],  # SoB_space
            df_events[key + '_SoB_space_ss'],  # SoB_space_ss
            df_events[key + '_SoB_energy'],  # SoB_energy
        ),
    )
    
    # save fitted ns and ts to the dictionary
    res_dict[key] = {
        'ns': result.x[0],
        'ts': -result.fun,
    } 
    print('  ns: {:3.1f} | ts: {:3.2f}'.format(result.x[0], -result.fun))


## Compute Significance

In order to compute the significance of rejecting the null-hypothesis, we need to compare the computed test-statistic value against background trials. Background trials are obtained by randomizing the experimental data by scrambling the right ascension value of the events. 
The test-statistic distribution of the background trials is provided in the data release via two files that define the bin edges and the bin heights of each bin.

In [ ]:
df_bins = pd.read_csv(os.path.join(data_dir, 'bkg_trials_binning.csv'))
df_values = pd.read_csv(os.path.join(data_dir, 'bkg_trials_values.csv'))
df_values

The function `get_significance_from_hist` computes the significance to reject the null-hypothesis for a given test-statistic value by evaluating the fraction of background trials with an equally large or larger test-statist value. This fraction corresponds to the probability of falsely rejecting the null-hypothesis and is therefore considered as the `p-value`. We can then convert the p-value to "multiples of sigma" by checking what number of standard deviations such a p-value would correspond to for a normal distribution.

In [ ]:
def get_significance_from_hist(ts, bin_edges, bin_values):
    """Compute significance for a given test-statistic
    
    Parameters
    ----------
    ts : float
        The test-statistic value for which the significance will be calculated.
    bin_edges : array_like
        The bin edges for the background trial histogram.
    bin_values : array_like
        The bin heights for the background trial histogram.
    
    Returns
    -------
    float
        The significance of rejecting the null hypothesis for the given
        test-statistic value based on the background trials provided
        via `bin_edges` and `bin_values`
    """
    # get index for which is valid:
    # edge[i-1] < ts <= edge[i]
    # this correspdonds to the i-th entry in bin_values
    index = np.searchsorted(bin_edges, ts)
    
    # get number of trials with larger ts values
    n_larger = np.sum(bin_values[index:])
    pval = 1.*n_larger/np.sum(bin_values)
    nsigma = stats.norm.isf(pval)
    
    return pval, nsigma

Now we will compute the significance for each of the tested GP models:

In [ ]:
for key in ['pi0', 'kra5', 'kra50']:
    pval, nsigma = get_significance_from_hist(
        ts=res_dict[key]['ts'],
        bin_edges=df_bins[key].values,
        bin_values=df_values[key].values,
    )
    res_dict[key]['nsigma'] = nsigma
    print('{:3.2f} sigma | GP model: {}'.format(nsigma, key))

## Convert $\hat{n}_s$ to Flux

In order to compute the flux for each of the GP models, the fitted number of signal events needs to be converted by utilizing the effective area of the event sample.

We'll start by loading the effective area, which is provided as a 2D array with the first axis in sin(dec) and the second axis in energy (units of GeV). The binning edges are provided in separate files.

In [ ]:
effa_values = np.loadtxt(os.path.join(data_dir, 'effa_values.csv'), delimiter=',')
print('Shape of 2D effective area:', effa_values.shape)

effa_bins_sindec = np.loadtxt(os.path.join(data_dir, 'effa_bins_sindec.csv'), delimiter=',')
print('Number of provided bin edges in sin(dec):', len(effa_bins_sindec))

effa_bins_energy = np.loadtxt(os.path.join(data_dir, 'effa_bins_energy.csv'), delimiter=',')
print('Number of provided bin edges in energy:', len(effa_bins_energy))


In addition to the effective area, the galactic plane templates are also required. The KRA-$\gamma$ templates are provided on zenodo at this location: https://zenodo.org/record/7070823 and the $\pi^0$ is located here: https://galprop.stanford.edu/PaperIISuppMaterial/SS_Z4_R20_T150_C5.tar .
If downloaded from the provided links and placed in the defined `data_dir`, these can be loaded via:

In [ ]:
template_kra5, template_kra5_nu, template_kra5_nubar, template_ebins5 = np.load(
    os.path.join(data_dir, 'KRA-gamma_5PeV_maps_energies.tuple.npy'), 
    allow_pickle = True, 
    encoding='latin1',
)
template_kra50, template_kra50_nu, template_kra50_nubar, template_ebins50 = np.load(
    os.path.join(data_dir, 'KRA-gamma_maps_energies.tuple.npy'), 
    allow_pickle = True, 
    encoding='latin1',
)
template_pi0 = np.load(
    os.path.join(data_dir, 'Fermi-LAT_pi0_map.npy'), 
    allow_pickle=True, 
    encoding='latin1',
)


#### Acceptance Correction for $\pi^0$ model

The $\pi^0$ template is provided as a healpix map normalized over the sky. The units are $\mathrm{sr}^-1$. The flux at each pixel is then simply the sky-integrated power-law flux multiplied by this spatial PDF. When combined with the effective area, the total number of expected events (`total_acceptance`) can be computed for a given reference normalization. The ratio to the number of observed signal events is then used to convert to a flux. This conversion is outlined in the function `acceptance_correct_pi0`.

In [ ]:
def acceptance_correct_pi0(
            ns, template, eff_area, sindec_bins, energy_bins,
            livetime=304047105.0735066,
        ):
    """Convert a number of signal events to the corresponding flux

    Parameters
    ----------
    ns : array_like
        The number of signal events that will be converted to the
        corresponding flux.
    template : array_like
        The spatial template in units of sr^-1.
        Shape: [npix]
    eff_area : array_like
        The effective area binned in sin(dec) (unitless) along first
        dimension and and in energy (in GeV) along the second axis.
        shape: [n_bins_sindec, n_bins_energy]
    sindec_bins : array_like
        The bin edges along the first dimension corresponding to sin(dec).
        Shape: [n_bins_sindec + 1]
    energy_bins : TYPE
        The bin edges along the second dimension corresponding to
        the energy in GeV.
        Shape: [n_bins_energy + 1]
    livetime : float
        The livetime of the dataset in seconds.
        For the 10-year cascade dataset, the livetime corresponds to
        304047105.0735066 seconds, which is the default value.

    Returns
    -------
    array_like
        The flux in terms of E^2 dN/dE at 100 TeV in units
        of TeV cm^-2 s^-1.
    """

    npix = len(template)
    nside = hp.npix2nside(npix)
    
    # First we need to construct Phi(sindec, energy)
    # in units of GeV^-1 cm^-2 s^-1 sr^-1.
    # We will do this by splitting Phi into the spatial term (units of sr^-1)
    # and the energy term (units of GeV^-1 cm^-2 s^-1).
    # We can do this splitting for the pi0 model because the energy spectrum
    # is assumed not to depend on the location in the sky
    theta, phi = hp.pix2ang(nside, np.r_[:npix])
    pix_dec = np.pi/2. - theta
    pix_ra = phi
    pix_sindec = np.sin(pix_dec)

    # Phi(sindec), shape: [n_sindec], units of sr^-1
    phi_sindec = np.zeros(len(sindec_bins)-1)

    # walk through each declination band
    for i, sindec_max in enumerate(sindec_bins[1:]):
        sindec_min = sindec_bins[i]

        # get all pixels belonging to this dec band
        mask_pixels = np.logical_and(
            pix_sindec >= sindec_min,
            pix_sindec < sindec_max,
        )
        phi_sindec[i] = np.sum(template[mask_pixels]) 

    # Sky-integrated, per-flavor (nu + nubar) flux
    # Choose an arbitrary flux normalization to scale
    norm = 1e-18  # in units of GeV^-1 cm^-2 s^-1
    e0 = 1e5  # 100 TeV in units of GeV
    gamma = 2.7

    # compute average energy in each energy bin
    energy_avg = (
        (-gamma + 1) / (-gamma + 2) *
        (energy_bins[1:]**(-gamma+2) - energy_bins[:-1]**(-gamma+2)) /
        (energy_bins[1:]**(-gamma+1) - energy_bins[:-1]**(-gamma+1))
    )

    # Phi(energy) in units of GeV^-1 cm^-2 s^-1 sr^-1
    # shape: [n_energy]
    phi_e = norm * (energy_avg / e0) ** (-gamma)

    # Phi(sindec, energy)
    # shape: [n_sindec, n_energy],  GeV^-1 cm^-2 s^-1 sr^-1
    phi = phi_e[np.newaxis] * phi_sindec[:, np.newaxis]

    # "integrate" over energy bin width
    # shape: [n_sindec, n_energy], cm^-2 s^-1 sr^-1
    phi *= np.diff(energy_bins)[np.newaxis]

    # "integrate" over solid angle
    # shape: [n_pix, n_bins], s^-1 * cm^-2
    phi *= hp.nside2pixarea(nside)

    # Now we can compute the total number of expected events
    # for the given flux.
    # eff_area is in units of cm^-2 and livetime in units of s
    total_acceptance = livetime * np.sum(eff_area * phi)

    # we can then compute the ratio of the number of signal events versus
    # the number of events one would have expected based on the previously
    # defined flux. This can then be used to scale the arbitrary flux
    # normalization that we chose as a reference point.
    model_norm = ns / total_acceptance

    # compute E^2 dN/dE at 100 TeV in units of TeV cm^-2 s^-1
    norm_tev = norm * 1e3  # in units of TeV^-1 cm^-2 s^-1
    norm_E2_tev = norm_tev * (100)**2
    return model_norm*norm_E2_tev

We are now almost ready to convert the best-fit $\hat{n}_s$ to the corresponding flux in terms of $E^2 \frac{dN}{dE}$ at 100 TeV in units of $\mathrm{TeV}^{-1} \mathrm{cm}^{-2} \mathrm{s}^{-1}$.
The last remaining step is to bias-correct the best-fit $\hat{n}_s$. In case of mis-modeling of the PDF ratios, the fitted number of signal events may have a systematic shift and thus not properly recover the true underlying signal events. This bias can be evaluated from MC injection trials. A number of signal trials are performed in which a given number of signal events $n_\mathrm{inj}$ are injected on top of the scrambled background distribution. For each of these trials, the analysis is run and the best-fit $\hat{n}_s$ is recorded. The bias can then be evaluated by comparing the median of the best-fit $\hat{n}_s$ distribution in comparison to the true number of injected signal events $n_\mathrm{inj}$. 


In [ ]:
# load data from signal trials
df_ns_bias_pi0 = pd.read_csv(os.path.join(data_dir, 'ns_bias_pi0.csv'))

# plot the ns-bias together with the correction function
fig, ax = plt.subplots()
ax.plot(df_ns_bias_pi0['n_inj'], df_ns_bias_pi0['median_ns'], label=r'$\pi^0$')
ax.set_title('Model: {}'.format(key))
ax.set_xlabel('$n_\mathrm{inj}$')
ax.set_ylabel('$\hat{n}_s$')
max_val = np.max(df_ns_bias_pi0['median_ns'])
ax.plot((0., max_val), (0., max_val), ls='--', color='0.7')
ax.legend()


As seen above, a slight bias is visible: the fitted number of events $\hat{n}_s$ tend to overestimate the true injected number of events. We will correct for this bias in order to obtain a better estimate of the true number of signal events by defining a bias-correction function `bias_corr_func_pi0`.

In [ ]:
# fit a spline function to the value pairs
# we will use this later to bias-correct the best-fit ns
sorted_idx = np.argsort(df_ns_bias_pi0['median_ns'])
bias_corr_func_pi0 = UnivariateSpline(
    x=df_ns_bias_pi0['median_ns'][sorted_idx], 
    y=df_ns_bias_pi0['n_inj'][sorted_idx], 
    # choose an appropriate smoothing for the spline
    s=len(df_ns_bias_pi0['n_inj'])*500,
)

print('Bias corrected ns: {:3.1f}'.format(bias_corr_func_pi0(res_dict['pi0']['ns'])))

Now we can convert the bias-corrected $\hat{n}_s$ to the corresponding flux in terms of $E^2 \frac{dN}{dE}$ at 100 TeV in units of $\mathrm{TeV}^{-1} \mathrm{cm}^{-2} \mathrm{s}^{-1}$.

In [ ]:
E2dNdE_pi0 = acceptance_correct_pi0(
    ns=bias_corr_func_pi0(res_dict['pi0']['ns']),
    template=template_pi0,
    eff_area=effa_values,
    sindec_bins=effa_bins_sindec,
    energy_bins=effa_bins_energy,
)
E2dNdE_pi0

The remaining <1% difference to the flux value stated in the paper is due to the necessary discretization of the effective area, while the analysis directly uses MC simulation.

## ToDo

- add acceptance correction for KRA-$\gamma$ models


## Plot flux (simplified version of Figure 5)

In [ ]:
def powerlaw(energy, E2dNdE_at_100TeV, gamma=2.7):
    """Powerlaw flux

    Parameters
    ----------
    energy : array_like
        The energy in GeV at which to evaluate the powerlaw flux.
    E2dNdE_at_100TeV : array_like
        The powerlaw normalization in terms of E^2 dN/dE at 100 TeV
        in units of TeV^-1 cm^-2 s^-1.
    gamma : float, optional
        The spectral index.

    Returns
    -------
    array_like
        The powerlaw flux evaluated at the provided energies in terms
        of E^2 dN/dE in units of GeV^-1 cm^-2 s^-1.
    """
    e0 = 1e5  # 100 TeV

    # get normalization in dN/dE in units of GeV^-1 cm^-2 s^-1
    norm = E2dNdE_at_100TeV / (e0)**2 * 1e3
    dNdE = norm * (energy/e0)**(-gamma)
    return dNdE * energy**2

energies = np.logspace(np.log10(500), 6.5, 100)
fig, ax = plt.subplots()
ax.plot(energies, powerlaw(energies, E2dNdE_at_100TeV=E2dNdE_pi0), label=r'$\pi^0$ best-fit $\nu$ flux')
ax.set_ylabel(r'E$_{\nu}^2$ $\frac{dN}{dE_\nu}$ [GeV s$^{-1}$ cm$^{-2}$]')
ax.set_xlabel(R'E$_\nu$ [GeV]')
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()
